# Prepration

- In order to train the Yolo v3 model, we will need to compile `darknet` which is an open source neural network framework written in C and CUDA. 
- Once the `darknet` is compiled, we will download the exported data set from HyperLabel


**Note**: Before starting to work, change the runtime to use the GPU.

If you don't have any dataset or pretrained model (weight) for that dataset, you can use our demo data set and trained weight.

Copy the following files to your Google drive (create a new folder called `ml` in your Google drive and put the files in there)

- [Dataset](https://drive.google.com/file/d/1wxHNFR2xXx6wJMJR2ey2NP_rWFJNs8-I/view?usp=sharing)
- [Pre trained weight](https://drive.google.com/file/d/1-5iEJhBgwRGBZ_ZBfO6Juo2tF6lGa4b6/view?usp=sharing)


### Clone darknet

In [0]:
!git clone https://github.com/AlexeyAB/darknet

In [0]:
from google.colab import drive
drive.mount('/content/drive')

### Compile darknet

In [0]:
%cd darknet
!sed -i 's/GPU=0/GPU=1/g' Makefile
!cat Makefile
!make

### Download the data set exported from HyperLabel


In [0]:
import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
  
DOWNLOAD_LOCATION = '/content/darknet/data/'
DRIVE_DATASET_FILE = '/content/gdrive/My Drive/Colab Notebooks/yolo-dataset.zip' #adjust path/name of dataset which is in your G-drive

shutil.copy(DRIVE_DATASET_FILE, DOWNLOAD_LOCATION)

print('Successfully downloaded the dataset')

### Unzip the dataset

In [0]:
!unzip data/yolo-dataset.zip -d data/ # adjust the dataset filename which you have downloaded from Google drive

##Training the model

- Download the initial weight
- Train the model

### Download initial pre-trained weights for the convolutional layers (If you have already trained and saved the weights to your Google drive, you can skip this)

In [0]:
!wget https://pjreddie.com/media/files/darknet53.conv.74

### Set Yolo configurations

We need to set configurations for Yolo in order to properly train. There are few settings which we need to change in the default yolov3.cfg file.

- batch
- subdivisions (if you get memory out error, increase this 16, 32 or 64)
- max_batches (it should be classes*2000)
- steps (it should be 80%, 90% of max_batches)
- classes (the number of classes which you are going to train)
- filters (the value for filters can be calculated using (classes + 5)x3 )

Change the values below as per your requirement

In [0]:
!sed -i 's/batch=1/batch=64/g' cfg/yolov3.cfg
!sed -i 's/subdivisions=1/subdivisions=32/g' cfg/yolov3.cfg
!sed -i 's/max_batches = 500200/max_batches = 2000/g' cfg/yolov3.cfg
!sed -i 's/steps=400000,450000/steps=1600,1800/g' cfg/yolov3.cfg
!sed -i 's/classes=80/classes=1/g' cfg/yolov3.cfg
!sed -i 's/filters=255/filters=18/g' cfg/yolov3.cfg
!cat cfg/yolov3.cfg

**Now we are moving forward with the training part. If you have already trained the model, you can fetch them from Google drive and skip the training part.**

In [0]:
# Only run this cell, if you have already trained the model and have weights and backup files in your Google drive
# (Optional) Download the pretrained weight from Google drive

import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
  
BACKUP_FOLDER = '/content/darknet/backup'
DRIVE_YOLO_BACKUP = '/content/gdrive/My Drive/ml/yolov3_last.weights'

shutil.copy(DRIVE_YOLO_BACKUP, BACKUP_FOLDER)

print('Successfully fetched the pretrained files for Yolo from Google drive')

### Start the training

It will take a long time to complete, so sit back and relax

Note: If during training you see nan values for avg (loss) field - then training goes wrong, but if nan is in some other lines - then training goes well.

- file yolo-obj_last.weights will be saved to the darknet/backup for each 100 iterations
- file yolo-obj_xxxx.weights will be saved to the darknet/backup for each 1000 iterations
- After each 100 iterations if you want, you can stop and later start training from this point. For example, after 2000 iterations you can stop training, and later just start training using: darknet detector train data/obj.data yolov3.cfg backup/yolo-obj_2000.weights

In [0]:
# use the line below to train a fresh model
!./darknet detector train /content/darknet/data/obj.data cfg/yolov3.cfg darknet53.conv.74

# use the line below to retrain your previous saved weight
#!./darknet detector train data/obj.data cfg/yolov3.cfg backup/yolov3_last.weights

In [0]:
# Once you have trained your model, you can save them to your Google drive. So that next time, you don't need to retrain
# This step is optional, you can skip it if you want
import os.path
import shutil
from google.colab import drive

if not os.path.exists('/content/gdrive'):
  drive.mount('/content/gdrive')
  
YOLO_BACKUP = '/content/darknet/backup/yolov3_1000.weights' #adjust the backup file name or keep it default
DRIVE_DIR = '/content/gdrive/My Drive/ALPR' #adjust path in your Google drive, or keep it default

shutil.copy(YOLO_BACKUP, DRIVE_DIR)

print('Saved training data to drive at: ' + DRIVE_DIR)

## Running predictions

In [0]:
!./darknet detector test data/obj.data cfg/yolov3.cfg backup/yolov3_last.weights /content/darknet/data/images/055.jpg

### Display result

In [0]:
def display_image(file_path = '/content/darknet/predictions.jpg'):
    import cv2
    import matplotlib.pyplot as plt
    import os.path

    if os.path.exists(file_path):
      img = cv2.imread(file_path)
      show_img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB) 
      plt.imshow(show_img)
    else:
      print('failed to open file')
    
display_image()